In [107]:
import json
from elasticsearch import Elasticsearch

# Connect to Elasticsearch
es = Elasticsearch(
    "https://localhost:9200",
    ca_certs=".\certs\ca\ca.crt",
    basic_auth=("elastic", "hammasir"),
    verify_certs=False,
    ssl_show_warn=False)
es.info()

<>:7: SyntaxWarning: invalid escape sequence '\c'
<>:7: SyntaxWarning: invalid escape sequence '\c'
C:\Users\SDR\AppData\Local\Temp\ipykernel_16148\1702532750.py:7: SyntaxWarning: invalid escape sequence '\c'
  ca_certs=".\certs\ca\ca.crt",


ObjectApiResponse({'name': 'es01', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'toAtBBXbQKm8i7sXCH722Q', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [32]:
with open('../data/processed/base_dataset.json') as f:
    json_data = json.load(f)

In [108]:
import pandas as pd
data = pd.read_csv("../data/processed/base_dataset.csv")

In [117]:
import ast
def safe_literal_eval(val):
    try:
        if isinstance(val, str):
            return ast.literal_eval(val)
        else:
            return val  # If it's not a string, return the original value
    except (ValueError, SyntaxError):
        # Return None or a default value if evaluation fails
        return None

In [118]:
data["clinic"] = data["clinic"].apply(safe_literal_eval)  

In [119]:
import numpy as np
data = data.replace(np.nan, None)

In [120]:
mappings = {
        "properties": {
            "gender": {"type": "text"},
            "expertise": {"type": "text"},
            "title": {"type": "text"},
            "star": {"type": "float"},
            "rates_count": {"type": "integer"},
            "number_of_visits": {"type": "integer"},
            "view": {"type": "text"},
            "insurances": {"type": "text"},
            "experience": {"type": "integer"},
            "doctor_encounter": {"type": "float"},
            "explanation_of_issue": {"type": "float"},
            "quality_of_treatment": {"type": "float"},
            "comments_count": {"type": "integer"},
            "waiting_time": {"type": "float"},
            "clinic": {"type": "object"},
    }
}

In [121]:
es.indices.delete(index="doctors")

ObjectApiResponse({'acknowledged': True})

In [122]:
es.indices.create(index="doctors", mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'doctors'})

In [123]:
for i, row in data.iterrows():
    doc = {
        "expertise": row["display_expertise"],
        "gender": row["gender"],
        "experience": row["experience"],
        "title": row["title"],
        "star": row["star"],
        "rates_count" : row["rates_count"],
        "number_of_visits": row["number_of_visits"],
        "view": row["view"],
        "insurances": row["insurances"],
        "doctor_encounter": row["doctor_encounter"],
        "explanation_of_issue": row["explanation_of_issue"],
        "quality_of_treatment": row["quality_of_treatment"],
        "comments_count": row["comments_count"],
        "waiting_time": row["waiting_time"],
        "clinic": row["clinic"]
    }

    es.index(index="doctors", id=i, document=doc)

In [124]:
es.indices.refresh(index="doctors")
es.cat.count(index="doctors", format="json")

ListApiResponse([{'epoch': '1725281743', 'timestamp': '12:55:43', 'count': '16296'}])

In [125]:
es.search(index="doctors", q="ماما")

ObjectApiResponse({'took': 136, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 13.364021, 'hits': [{'_index': 'doctors', '_id': '10820', '_score': 13.364021, '_source': {'expertise': 'ماما', 'gender': 'F', 'experience': None, 'title': 'محبوبه رستگارراد', 'star': 5.0, 'rates_count': 1.0, 'number_of_visits': 1035.0, 'view': '1K', 'insurances': '[]', 'doctor_encounter': 5.0, 'explanation_of_issue': 5.0, 'quality_of_treatment': 5.0, 'comments_count': 1.0, 'waiting_time': 0.0, 'clinic': [{'city': 'مشهد', 'number': None, 'address': 'مشهد، بلوار حر، بین حر 7  و 9', 'province_name': 'خراسان رضوی'}]}}, {'_index': 'doctors', '_id': '10758', '_score': 9.523348, '_source': {'expertise': 'زنان _ مامایی _ ماما همراه _ ناباروری', 'gender': 'F', 'experience': None, 'title': 'صابری مفرد', 'star': 5.0, 'rates_count': 18.0, 'number_of_visits': 1457.0, 'view': '1K', 'insurances': '[]', 'doctor_encounte